In [1]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to C:\Users\Nathan
[nltk_data]     Kinalendele\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import numpy as np
import json
import glob

import re
import string

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim
from sklearn.manifold import MDS


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [74]:

def preprocess_text(text):
    # Remove the word "action"
   # Remove the words "action" and "github"
    text = re.sub(r'\b(?:action|github|actions)\b', '', text, flags=re.IGNORECASE)
    
    # Remove code snippets enclosed in <code> tags
    text = re.sub(r'<code>(.*?)</code>', '', text, flags=re.DOTALL)
    
    # Remove HTML tags
    text = re.sub('<[^<]+?>', '', text)
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove punctuation marks
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove leading/trailing white spaces
    text = text.strip()
    
    return text

def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
        texts = []
        for item in data:
            title = item["title"]
            body = item["body"]
            question_text = title + " " + body
            preprocessed_text = preprocess_text(question_text)
            texts.append(preprocessed_text)
        return texts
    
def write_data(file, data):
    with open(file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)
        
file_path = "nath.json"
corpus = load_data(file_path)
write_data("preprocessed_data.json", corpus)


In [75]:
data = load_data("nath.json")

print (data[0])

what are the differences between   and other ci tools like jenkins  announced an upcoming feature  

im positive on the benefits of ci tools like jenkins for automatic building or testing which   is aimed to be used for in the future

having a repository on  and using an external ci tool has the huge benefit of allowing to move the repository to another git repository platform or even local without rewriting of the whole ci process with   youre more or less tied to the  ecosystem

i assume the integration of s  will be more fluent in the native environment but are there any other advantages or disadvantages besides that


In [76]:

def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
                
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0])

difference other ci tool jenkin announce upcoming feature m positive benefit tool jenkin automatic building testing aim use future have repository use external ci tool huge benefit allow move repository git repository platform even local rewrite whole ci process re more less tie ecosystem assume integration s more fluent native environment other advantage disadvantage


In [77]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess( text, deacc=True)
        final.append(new)
        
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0])

['difference', 'other', 'ci', 'tool', 'jenkin', 'announce', 'upcoming', 'feature', 'positive', 'benefit', 'tool', 'jenkin', 'automatic', 'building', 'testing', 'aim', 'use', 'future', 'have', 'repository', 'use', 'external', 'ci', 'tool', 'huge', 'benefit', 'allow', 'move', 'repository', 'git', 'repository', 'platform', 'even', 'local', 'rewrite', 'whole', 'ci', 'process', 're', 'more', 'less', 'tie', 'ecosystem', 'assume', 'integration', 'more', 'fluent', 'native', 'environment', 'other', 'advantage', 'disadvantage']


In [81]:
bigrams_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigrams_phrases[data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigrams_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

def make_trigram(texts):
    return [trigram[bigram[doc]] for doc in texts]

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigram(data_bigrams)

print(data_bigrams_trigrams[0])

['difference', 'other', 'ci', 'tool', 'jenkin', 'announce', 'upcoming', 'feature', 'positive', 'benefit', 'tool', 'jenkin', 'automatic', 'building', 'testing', 'aim', 'use', 'future', 'have', 'repository', 'use', 'external', 'ci', 'tool', 'huge', 'benefit', 'allow', 'move', 'repository', 'git', 'repository', 'platform', 'even', 'local', 'rewrite', 'whole', 'ci', 'process', 're', 'more', 'less', 'tie', 'ecosystem', 'assume', 'integration', 'more', 'fluent', 'native', 'environment', 'other', 'advantage', 'disadvantage']


In [82]:
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
print (corpus[0])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]
    
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow
    

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 3), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 3), (34, 1), (35, 1), (36, 1), (37, 3), (38, 1), (39, 2), (40, 1)]


In [83]:
import json

def get_corpus_size(json_file):
    with open(json_file, 'r', encoding='utf-8') as file:
        data = json.load(file)

    num_documents = len(data)
    total_tokens = 0

    for post in data:
        # Assuming the "body" and "title" fields contain the text of the document
        doc_text = post['body'] + " " + post['title']
        words = doc_text.split()  # Split the document into words/tokens
        total_tokens += len(words)

    return num_documents, total_tokens

# Replace 'your_json_file.json' with the path to your JSON file
json_file_path = 'nath.json'
num_documents, total_tokens = get_corpus_size(json_file_path)

print(f"Number of documents in the corpus: {num_documents}")
print(f"Total number of tokens in the corpus: {total_tokens}")


Number of documents in the corpus: 6149
Total number of tokens in the corpus: 1198347


In [17]:
import time
import gensim
from gensim.models import LdaModel
from gensim import corpora

# Preprocessed texts
texts = data_bigrams_trigrams  # Your preprocessed texts here

# Create a dictionary from the texts
dictionary = corpora.Dictionary(data_bigrams_trigrams)

# Create a corpus (bag of words representation)
corpus = [dictionary.doc2bow(text) for text in texts]

# Define the ranges for the parameters
num_topics_range = range(5, 15)  # Number of topics: 2 to 50
num_iterations_range = range(100, 500, 100)  # Number of iterations: 10 to 500 (step size: 10)
chunk_size_range = range(1000, 2000, 100)  # Chunk size: 10 to 200 (step size: 10)
passes_range = range(100, 200, 10)  # Number of passes: 1 to 100

# Initialize variables for tracking the optimal parameters and coherence score
optimal_params = {}
max_coherence_score = 0

# Start measuring the overall execution time
start_time = time.time()

# Iterate over the parameter ranges and train LDA models
for num_topics in num_topics_range:
    for num_iterations in num_iterations_range:
        for chunk_size in chunk_size_range:
            for passes in passes_range:
                # Start measuring the execution time for each LDA model
                model_start_time = time.time()

                lda_model = LdaModel(corpus=corpus,
                                     id2word=dictionary,
                                     num_topics=num_topics,
                                     iterations=num_iterations,
                                     chunksize=chunk_size,
                                     random_state=100)
                # Calculate coherence score
                coherence_model = gensim.models.CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
                coherence_score = coherence_model.get_coherence()

                # Check if current coherence score is higher than previous max
                if coherence_score > max_coherence_score:
                    max_coherence_score = coherence_score
                    optimal_params = {
                        'num_topics': num_topics,
                        'num_iterations': num_iterations,
                        'chunk_size': chunk_size,
                        'passes': passes
                    }

                # Calculate the elapsed time for the current LDA model
                model_elapsed_time = time.time() - model_start_time
                print("Elapsed Time for Model (Topics: {}, Iterations: {}, Chunk Size: {}, Passes: {}): {:.2f} seconds, Coherence Score: {:.4f}".format(num_topics, num_iterations, chunk_size, passes, model_elapsed_time, coherence_score))

# Calculate the overall elapsed time
elapsed_time = time.time() - start_time

# Print the optimal parameters and coherence score
print("Optimal Parameters:")
print("Number of Topics:", optimal_params['num_topics'])
print("Number of Iterations:", optimal_params['num_iterations'])
print("Chunk Size:", optimal_params['chunk_size'])
print("Passes:", optimal_params['passes'])
print("Coherence Score:", max_coherence_score)
print("Overall Execution Time: {:.2f} seconds".format(elapsed_time))


Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1000, Passes: 100): 12.69 seconds, Coherence Score: 0.3650
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1000, Passes: 110): 11.65 seconds, Coherence Score: 0.3650
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1000, Passes: 120): 12.29 seconds, Coherence Score: 0.3650
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1000, Passes: 130): 14.05 seconds, Coherence Score: 0.3650
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1000, Passes: 140): 22.04 seconds, Coherence Score: 0.3650
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1000, Passes: 150): 12.84 seconds, Coherence Score: 0.3650
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1000, Passes: 160): 15.16 seconds, Coherence Score: 0.3650
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1000, Passes: 170): 21.48 seconds, Coherence Score: 0.3650
Elapsed Time for

Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1600, Passes: 170): 12.82 seconds, Coherence Score: 0.3531
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1600, Passes: 180): 12.51 seconds, Coherence Score: 0.3531
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1600, Passes: 190): 12.39 seconds, Coherence Score: 0.3531
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1700, Passes: 100): 12.82 seconds, Coherence Score: 0.3694
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1700, Passes: 110): 12.75 seconds, Coherence Score: 0.3694
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1700, Passes: 120): 12.94 seconds, Coherence Score: 0.3694
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1700, Passes: 130): 13.34 seconds, Coherence Score: 0.3694
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1700, Passes: 140): 14.49 seconds, Coherence Score: 0.3694
Elapsed Time for

Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1300, Passes: 140): 14.00 seconds, Coherence Score: 0.3653
Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1300, Passes: 150): 14.84 seconds, Coherence Score: 0.3653
Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1300, Passes: 160): 12.68 seconds, Coherence Score: 0.3653
Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1300, Passes: 170): 14.35 seconds, Coherence Score: 0.3653
Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1300, Passes: 180): 14.18 seconds, Coherence Score: 0.3653
Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1300, Passes: 190): 14.02 seconds, Coherence Score: 0.3653
Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1400, Passes: 100): 13.44 seconds, Coherence Score: 0.3649
Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1400, Passes: 110): 13.82 seconds, Coherence Score: 0.3649
Elapsed Time for

Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1000, Passes: 110): 12.13 seconds, Coherence Score: 0.3647
Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1000, Passes: 120): 12.15 seconds, Coherence Score: 0.3647
Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1000, Passes: 130): 12.16 seconds, Coherence Score: 0.3647
Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1000, Passes: 140): 12.08 seconds, Coherence Score: 0.3647
Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1000, Passes: 150): 12.27 seconds, Coherence Score: 0.3647
Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1000, Passes: 160): 12.18 seconds, Coherence Score: 0.3647
Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1000, Passes: 170): 12.12 seconds, Coherence Score: 0.3647
Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1000, Passes: 180): 11.96 seconds, Coherence Score: 0.3647
Elapsed Time for

Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1600, Passes: 180): 14.34 seconds, Coherence Score: 0.3603
Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1600, Passes: 190): 14.49 seconds, Coherence Score: 0.3603
Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1700, Passes: 100): 14.36 seconds, Coherence Score: 0.3683
Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1700, Passes: 110): 14.25 seconds, Coherence Score: 0.3683
Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1700, Passes: 120): 13.92 seconds, Coherence Score: 0.3683
Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1700, Passes: 130): 14.17 seconds, Coherence Score: 0.3683
Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1700, Passes: 140): 14.32 seconds, Coherence Score: 0.3683
Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1700, Passes: 150): 14.04 seconds, Coherence Score: 0.3683
Elapsed Time for

...

Elapsed Time for Model (Topics: 14, Iterations: 100, Chunk Size: 1800, Passes: 150): 13.35 seconds, Coherence Score: 0.3313
Elapsed Time for Model (Topics: 14, Iterations: 100, Chunk Size: 1800, Passes: 160): 13.07 seconds, Coherence Score: 0.3313
Elapsed Time for Model (Topics: 14, Iterations: 100, Chunk Size: 1800, Passes: 170): 13.02 seconds, Coherence Score: 0.3313
Elapsed Time for Model (Topics: 14, Iterations: 100, Chunk Size: 1800, Passes: 180): 12.71 seconds, Coherence Score: 0.3313
Elapsed Time for Model (Topics: 14, Iterations: 100, Chunk Size: 1800, Passes: 190): 12.80 seconds, Coherence Score: 0.3313
Elapsed Time for Model (Topics: 14, Iterations: 100, Chunk Size: 1900, Passes: 100): 12.89 seconds, Coherence Score: 0.3322
Elapsed Time for Model (Topics: 14, Iterations: 100, Chunk Size: 1900, Passes: 110): 12.45 seconds, Coherence Score: 0.3322
Elapsed Time for Model (Topics: 14, Iterations: 100, Chunk Size: 1900, Passes: 120): 12.51 seconds, Coherence Score: 0.3322
Elapsed 

Elapsed Time for Model (Topics: 14, Iterations: 200, Chunk Size: 1500, Passes: 120): 13.00 seconds, Coherence Score: 0.3396
Elapsed Time for Model (Topics: 14, Iterations: 200, Chunk Size: 1500, Passes: 130): 12.78 seconds, Coherence Score: 0.3396
Elapsed Time for Model (Topics: 14, Iterations: 200, Chunk Size: 1500, Passes: 140): 12.83 seconds, Coherence Score: 0.3396
Elapsed Time for Model (Topics: 14, Iterations: 200, Chunk Size: 1500, Passes: 150): 13.53 seconds, Coherence Score: 0.3396
Elapsed Time for Model (Topics: 14, Iterations: 200, Chunk Size: 1500, Passes: 160): 12.81 seconds, Coherence Score: 0.3396
Elapsed Time for Model (Topics: 14, Iterations: 200, Chunk Size: 1500, Passes: 170): 12.94 seconds, Coherence Score: 0.3396
Elapsed Time for Model (Topics: 14, Iterations: 200, Chunk Size: 1500, Passes: 180): 12.82 seconds, Coherence Score: 0.3396
Elapsed Time for Model (Topics: 14, Iterations: 200, Chunk Size: 1500, Passes: 190): 12.65 seconds, Coherence Score: 0.3396
Elapsed 

Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1100, Passes: 190): 13.03 seconds, Coherence Score: 0.3497
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1200, Passes: 100): 12.67 seconds, Coherence Score: 0.3396
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1200, Passes: 110): 12.88 seconds, Coherence Score: 0.3396
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1200, Passes: 120): 12.88 seconds, Coherence Score: 0.3396
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1200, Passes: 130): 15.80 seconds, Coherence Score: 0.3396
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1200, Passes: 140): 12.98 seconds, Coherence Score: 0.3396
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1200, Passes: 150): 12.86 seconds, Coherence Score: 0.3396
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1200, Passes: 160): 12.93 seconds, Coherence Score: 0.3396
Elapsed 

Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1800, Passes: 160): 13.88 seconds, Coherence Score: 0.3342
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1800, Passes: 170): 13.91 seconds, Coherence Score: 0.3342
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1800, Passes: 180): 13.78 seconds, Coherence Score: 0.3342
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1800, Passes: 190): 13.97 seconds, Coherence Score: 0.3342
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1900, Passes: 100): 13.55 seconds, Coherence Score: 0.3347
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1900, Passes: 110): 13.63 seconds, Coherence Score: 0.3347
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1900, Passes: 120): 13.92 seconds, Coherence Score: 0.3347
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1900, Passes: 130): 13.98 seconds, Coherence Score: 0.3347
Elapsed 

Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1500, Passes: 130): 13.01 seconds, Coherence Score: 0.3243
Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1500, Passes: 140): 13.22 seconds, Coherence Score: 0.3243
Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1500, Passes: 150): 13.40 seconds, Coherence Score: 0.3243
Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1500, Passes: 160): 13.40 seconds, Coherence Score: 0.3243
Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1500, Passes: 170): 13.08 seconds, Coherence Score: 0.3243
Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1500, Passes: 180): 13.02 seconds, Coherence Score: 0.3243
Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1500, Passes: 190): 13.14 seconds, Coherence Score: 0.3243
Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1600, Passes: 100): 14.30 seconds, Coherence Score: 0.3357
Elapsed 

In [90]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 

                                            id2word=id2word, 
                                            num_topics = 7,
passes = 100,
chunksize = 1100,
iterations=100,
alpha = 'auto',
eta = 'auto',
random_state = 100


                                            ) 


In [106]:
from gensim.models.ldamodel import LdaModel
from gensim import corpora

# Assuming you have already trained your LDA model and loaded your corpus and id2word dictionary
# lda_model = ...

# Load your test document (replace 'test_document' with your actual document)
test_document = "This is a test document for topic modeling."

# Preprocess the test document and convert it to a bag-of-words representation
test_bow = id2word.doc2bow(test_document.split())

# Get the probability distribution over topics for the test document
topic_probabilities = lda_model.get_document_topics(test_bow)

# Calculate the sum of all topic probabilities
sum_probabilities = sum(probability for _, probability in topic_probabilities)

# Print the percentage of each topic for the test document
for topic, probability in topic_probabilities:
    percentage = (probability / sum_probabilities) * 100
    topic_keywords = lda_model.show_topic(topic)  # Get the top words for the topic
    keyword_string = ', '.join([word for word, _ in topic_keywords])
    print(f"Topic {topic}: Probability: {probability:.4f} ({percentage:.2f}% of total), Keywords: {keyword_string}")


Topic 0: Probability: 0.2289 (22.89% of total), Keywords: file, build, use, project, create, way, want, need, find, directory
Topic 1: Probability: 0.0250 (2.50% of total), Keywords: docker, image, container, build, cache, dockerfile, runner, push, service, pull
Topic 2: Probability: 0.0352 (3.52% of total), Keywords: package, version, release, tag, publish, dependency, install, artifact, instal, download
Topic 3: Probability: 0.2071 (20.71% of total), Keywords: workflow, branch, push, trigger, job, commit, run, pull_request, check, merge
Topic 4: Probability: 0.0303 (3.03% of total), Keywords: deploy, app, azure, deployment, service, function, terraform, application, aw, page
Topic 5: Probability: 0.4161 (41.61% of total), Keywords: run, error, try, test, work, get, fail, here, use, workflow
Topic 6: Probability: 0.0575 (5.75% of total), Keywords: secret, use, access, value, repository, private, repo, key, variable, environment_variable


In [107]:
from gensim.models.ldamodel import LdaModel
from gensim import corpora

# Assuming you have already trained your LDA model and loaded your corpus and id2word dictionary
# lda_model = ...

# Assuming you have a corpus of documents
# corpus = ...

# Get the number of topics from the LDA model
num_topics = lda_model.num_topics

# Initialize a list to store the count of documents for each topic
topic_doc_count = [0] * num_topics

# Iterate through the corpus and count the documents associated with each topic
for document_bow in corpus:
    dominant_topic = max(lda_model[document_bow], key=lambda x: x[1])[0]
    topic_doc_count[dominant_topic] += 1

# Print the number of documents associated with each topic
for topic, doc_count in enumerate(topic_doc_count):
    print(f"Topic {topic}: Number of Documents: {doc_count}")


Topic 0: Number of Documents: 1474
Topic 1: Number of Documents: 147
Topic 2: Number of Documents: 127
Topic 3: Number of Documents: 852
Topic 4: Number of Documents: 140
Topic 5: Number of Documents: 2903
Topic 6: Number of Documents: 506


In [123]:
import json
from gensim.models import LdaModel
from gensim import corpora

# Step 1: Read data from the JSON file
with open('nath.json', 'r', encoding='utf-8') as json_file:
    data_list = json.load(json_file)

# Step 2: Assuming you have already trained your LDA model 'lda_model' and have your corpus available.
# (Please ensure you have the 'lda_model' and 'corpus' variables from your previous code.)

# Step 3: Associate topics with documents and find the dominant topic for each document
doc_topics = [lda_model.get_document_topics(doc) for doc in corpus]
dominant_topics = [max(doc_topics, key=lambda x: x[1])[0] for doc_topics in doc_topics]

# Step 4: Group data points (documents) based on their associated topics
topics_data = {topic_id: [] for topic_id in range(lda_model.num_topics)}
for doc_id, topic_id in enumerate(dominant_topics):
    topics_data[topic_id].append(data_list[doc_id])

# Step 5: Calculate averages for each topic
average_fav_count_per_topic = {}
average_score_per_topic = {}
average_view_count_per_topic = {}

# Step 6: Calculate the number of questions without an answer and without an accepted answer for each topic
num_questions_without_answer = {topic_id: 0 for topic_id in range(lda_model.num_topics)}
num_questions_without_accepted_answer = {topic_id: 0 for topic_id in range(lda_model.num_topics)}

total_documents = len(data_list)  # Total number of documents in the corpus

for topic_id, documents in topics_data.items():
    total_fav_count = 0
    total_score = 0
    total_view_count = 0
    num_documents = len(documents)

    for document in documents:
        # Assuming 'favorite_count', 'score', 'view_count', 'answer_count', and 'accepted_answer_id'
        # are attributes in each document dictionary
        total_fav_count += int(document.get('favorite_count', 0) or 0)
        total_score += int(document.get('score', 0) or 0)
        total_view_count += int(document.get('view_count', 0) or 0)

        if document.get('answer_count', 0) == 0:
            num_questions_without_answer[topic_id] += 1
        if document.get('accepted_answer_id') is None:
            num_questions_without_accepted_answer[topic_id] += 1

    average_fav_count_per_topic[topic_id] = total_fav_count / num_documents
    average_score_per_topic[topic_id] = total_score / num_documents
    average_view_count_per_topic[topic_id] = total_view_count / num_documents

# Print the keywords and document distribution for each topic
for topic_id in range(lda_model.num_topics):
    print(f"Topic {topic_id}:")
    keywords = lda_model.show_topic(topic_id, topn=10)  # Display top 10 keywords for each topic
    sum_probabilities = sum(weight for _, weight in keywords)
    for word, weight in keywords:
        print(f"{word}: {weight:.4f}")
    print(f"Sum of Probabilities for all keywords in this topic: {sum_probabilities:.4f}")
    num_documents = len(topics_data[topic_id])
    print(f"Number of documents associated with this topic: {num_documents}")
    print(f"Number of questions without an answer in this topic: {num_questions_without_answer[topic_id]}")
    print(f"Number of questions without an accepted answer in this topic: {num_questions_without_accepted_answer[topic_id]}")
    print()

# Step 7: Calculate overall probability of each topic
overall_topic_probability = {topic_id: len(documents) / total_documents * 100 for topic_id, documents in topics_data.items()}

# Step 8: Print the overall probabilities
for topic_id in range(lda_model.num_topics):
    print(f"Topic {topic_id}: Probability = {overall_topic_probability[topic_id]:.2f}%")
    print()

print("Average favorite_count per topic:", average_fav_count_per_topic)
print("Average score per topic:", average_score_per_topic)
print("Average view_count per topic:", average_view_count_per_topic)


Topic 0:
file: 0.0659
build: 0.0401
use: 0.0288
project: 0.0208
create: 0.0143
way: 0.0137
want: 0.0128
need: 0.0113
find: 0.0109
directory: 0.0104
Sum of Probabilities for all keywords in this topic: 0.2290
Number of documents associated with this topic: 1474
Number of questions without an answer in this topic: 0
Number of questions without an accepted answer in this topic: 892

Topic 1:
docker: 0.0840
image: 0.0807
container: 0.0537
build: 0.0297
cache: 0.0297
dockerfile: 0.0216
runner: 0.0140
push: 0.0134
service: 0.0113
pull: 0.0105
Sum of Probabilities for all keywords in this topic: 0.3486
Number of documents associated with this topic: 147
Number of questions without an answer in this topic: 0
Number of questions without an accepted answer in this topic: 99

Topic 2:
package: 0.0698
version: 0.0613
release: 0.0527
tag: 0.0436
publish: 0.0291
dependency: 0.0248
install: 0.0210
artifact: 0.0164
instal: 0.0125
download: 0.0104
Sum of Probabilities for all keywords in this topic: 0.

In [138]:
import json
from gensim.models import LdaModel
from gensim import corpora

# Step 1: Read data from the JSON file
with open('nath.json', 'r', encoding='utf-8') as json_file:
    data_list = json.load(json_file)

# Step 2: Assuming you have already trained your LDA model 'lda_model' and have your corpus available.
# (Please ensure you have the 'lda_model' and 'corpus' variables from your previous code.)

# Step 3: Associate topics with documents and find the dominant topic for each document
doc_topics = [lda_model.get_document_topics(doc) for doc in corpus]
dominant_topics = [max(doc_topics, key=lambda x: x[1])[0] for doc_topics in doc_topics]

# Step 4: Group data points (documents) based on their associated topics
topics_data = {topic_id: [] for topic_id in range(lda_model.num_topics)}
for doc_id, topic_id in enumerate(dominant_topics):
    topics_data[topic_id].append(data_list[doc_id])

# Step 5: Initialize dictionaries to store sums and counts for each topic
sum_fav_count_per_topic = {}
sum_score_per_topic = {}
sum_view_count_per_topic = {}
num_documents_per_topic = {}

# Step 6: Calculate sums and counts for each topic
for topic_id, documents in topics_data.items():
    sum_fav_count = 0
    sum_score = 0
    sum_view_count = 0
    num_documents = len(documents)

    for document in documents:
        fav_count = document.get('favorite_count')
        score = document.get('score')
        view_count = document.get('view_count')

        if fav_count is not None:
            sum_fav_count += int(fav_count)
        if score is not None:
            sum_score += int(score)
        if view_count is not None:
            sum_view_count += int(view_count)

    sum_fav_count_per_topic[topic_id] = sum_fav_count
    sum_score_per_topic[topic_id] = sum_score
    sum_view_count_per_topic[topic_id] = sum_view_count
    num_documents_per_topic[topic_id] = num_documents

# Print averages and topic keywords for each topic
for topic_id in range(lda_model.num_topics):
    num_documents = num_documents_per_topic[topic_id]
    if num_documents > 0:
        avg_fav_count = sum_fav_count_per_topic[topic_id] / num_documents
        avg_score = sum_score_per_topic[topic_id] / num_documents
        avg_view_count = sum_view_count_per_topic[topic_id] / num_documents

        print(f"Topic {topic_id}:")
        print(f"Topic Keywords: {', '.join([word for word, _ in lda_model.show_topic(topic_id)])}")
        print(f"Average favorite count: {avg_fav_count:.2f}")
        print(f"Average score: {avg_score:.2f}")
        print(f"Average view count: {avg_view_count:.2f}")
        print()

# ... (Rest of your code, if needed)


Topic 0:
Topic Keywords: file, build, use, project, create, way, want, need, find, directory
Average favorite count: 0.50
Average score: 2.31
Average view count: 1353.84

Topic 1:
Topic Keywords: docker, image, container, build, cache, dockerfile, runner, push, service, pull
Average favorite count: 0.45
Average score: 2.06
Average view count: 1217.19

Topic 2:
Topic Keywords: package, version, release, tag, publish, dependency, install, artifact, instal, download
Average favorite count: 0.53
Average score: 2.74
Average view count: 1339.01

Topic 3:
Topic Keywords: workflow, branch, push, trigger, job, commit, run, pull_request, check, merge
Average favorite count: 0.79
Average score: 4.16
Average view count: 2273.04

Topic 4:
Topic Keywords: deploy, app, azure, deployment, service, function, terraform, application, aw, page
Average favorite count: 0.28
Average score: 1.29
Average view count: 724.91

Topic 5:
Topic Keywords: run, error, try, test, work, get, fail, here, use, workflow
Av

In [139]:
# Step 1: Read data from the JSON file
with open('nath.json', 'r', encoding='utf-8') as json_file:
    data_list = json.load(json_file)

# Step 2: Assuming you have already trained your LDA model 'lda_model' and have your corpus available.
# (Please ensure you have the 'lda_model' and 'corpus' variables from your previous code.)

# Step 3: Associate topics with documents and find the dominant topic for each document
doc_topics = [lda_model.get_document_topics(doc) for doc in corpus]
dominant_topics = [max(doc_topics, key=lambda x: x[1])[0] for doc_topics in doc_topics]

# Step 4: Group data points (documents) based on their associated topics
topics_data = {topic_id: [] for topic_id in range(lda_model.num_topics)}
for doc_id, topic_id in enumerate(dominant_topics):
    topics_data[topic_id].append(data_list[doc_id])

# Step 5: Initialize dictionaries to store sums and counts for each topic
sum_fav_count_per_topic = {}
sum_score_per_topic = {}
sum_view_count_per_topic = {}
num_documents_per_topic = {}

# Step 6: Calculate sums and counts for each topic
for topic_id, documents in topics_data.items():
    sum_fav_count = 0
    sum_score = 0
    sum_view_count = 0
    num_documents = len(documents)

    for document in documents:
        fav_count = document.get('favorite_count')
        score = document.get('score')
        view_count = document.get('view_count')

        if fav_count is not None:
            sum_fav_count += int(fav_count)
        if score is not None:
            sum_score += int(score)
        if view_count is not None:
            sum_view_count += int(view_count)

    sum_fav_count_per_topic[topic_id] = sum_fav_count
    sum_score_per_topic[topic_id] = sum_score
    sum_view_count_per_topic[topic_id] = sum_view_count
    num_documents_per_topic[topic_id] = num_documents

# Calculate total averages for all questions/topics
total_questions = len(data_list)
total_avg_fav_count = sum(sum_fav_count_per_topic.values()) / total_questions
total_avg_score = sum(sum_score_per_topic.values()) / total_questions
total_avg_view_count = sum(sum_view_count_per_topic.values()) / total_questions

print("Average Attributes for All Topics/Questions:")
print(f"Total Questions: {total_questions}")
print(f"Total Average Favorite Count: {total_avg_fav_count:.2f}")
print(f"Total Average Score: {total_avg_score:.2f}")
print(f"Total Average View Count: {total_avg_view_count:.2f}")
print()


Average Attributes for All Topics/Questions:
Total Questions: 6149
Total Average Favorite Count: 0.50
Total Average Score: 2.64
Total Average View Count: 1569.31



In [135]:
import json
from gensim.models import LdaModel
from gensim import corpora

# Step 1: Read data from the JSON file
with open('nath.json', 'r', encoding='utf-8') as json_file:
    data_list = json.load(json_file)

# Step 2: Assuming you have already trained your LDA model 'lda_model' and have your corpus available.
# (Please ensure you have the 'lda_model' and 'corpus' variables from your previous code.)

# Step 3: Associate topics with documents and find the dominant topic for each document
doc_topics = [lda_model.get_document_topics(doc) for doc in corpus]
dominant_topics = [max(doc_topics, key=lambda x: x[1])[0] for doc_topics in doc_topics]

# Step 4: Group data points (documents) based on their associated topics
topics_data = {topic_id: [] for topic_id in range(lda_model.num_topics)}
for doc_id, topic_id in enumerate(dominant_topics):
    topics_data[topic_id].append(data_list[doc_id])

# Step 5: Initialize dictionaries to store counts for each topic
num_questions_without_accepted_answer = {}
num_questions_without_answer = {}

# Step 6: Calculate counts for each topic
for topic_id, documents in topics_data.items():
    num_without_accepted_answer = 0
    num_without_answer = 0

    for document in documents:
        answer_count = int(document.get('answer_count', 0))  # Convert to int
        accepted_answer_id = document.get('accepted_answer_id')

        if answer_count == 0:
            num_without_answer += 1
        if accepted_answer_id is None :
            num_without_accepted_answer += 1

    num_questions_without_answer[topic_id] = num_without_answer
    num_questions_without_accepted_answer[topic_id] = num_without_accepted_answer

# Print the number of questions without an accepted answer and without any answer for each topic
for topic_id in range(lda_model.num_topics):
    print(f"Topic {topic_id}:")
    print(f"Number of questions without an accepted answer: {num_questions_without_accepted_answer[topic_id]}")
    print(f"Number of questions without any answer: {num_questions_without_answer[topic_id]}")
    print()

# ... (Rest of your code, if needed)


Topic 0:
Number of questions without an accepted answer: 892
Number of questions without any answer: 392

Topic 1:
Number of questions without an accepted answer: 99
Number of questions without any answer: 47

Topic 2:
Number of questions without an accepted answer: 84
Number of questions without any answer: 41

Topic 3:
Number of questions without an accepted answer: 506
Number of questions without any answer: 221

Topic 4:
Number of questions without an accepted answer: 94
Number of questions without any answer: 42

Topic 5:
Number of questions without an accepted answer: 1796
Number of questions without any answer: 869

Topic 6:
Number of questions without an accepted answer: 289
Number of questions without any answer: 113



In [91]:
#warnings.filterwarnings('ignore', category=FutureWarning)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=10)
vis

C:\Users\Nathan Kinalendele\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\manifold\_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.088725  0.037386       1        1  34.683104
0     -0.107511 -0.247179       2        1  25.374265
3     -0.329401 -0.117886       3        1  13.256376
6     -0.255101  0.235271       4        1   9.818067
4      0.261198 -0.287618       5        1   5.775253
2      0.366481  0.046092       6        1   5.715319
1      0.153060  0.333934       7        1   5.377616, topic_info=         Term         Freq        Total Category  logprob  loglift
45       file  4693.000000  4693.000000  Default  10.0000  10.0000
127    deploy  1680.000000  1680.000000  Default   9.0000   9.0000
182  workflow  4216.000000  4216.000000  Default   8.0000   8.0000
99      build  3363.000000  3363.000000  Default   7.0000   7.0000
122    branch  1997.000000  1997.000000  Default   6.0000   6.0000
..        ...          ...          ...      ...      ...      ...
696    runner   209.429937   480.634873   Topic7  -4.2720   2.0922
688      pull   158.005249   294.509964   Topic7  -4.5537   2.3002
99      build   446.315137  3363.717071   Topic7  -3.5153   0.9031
289   service   169.510859   504.625636   Topic7  -4.4834   1.8320
159      push   201.002642  1766.550049   Topic7  -4.3130   0.7495

[132 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
41        4  0.925373       access
41        7  0.073662       access
594       1  0.120139          app
594       5  0.879295          app
218       1  0.320843  application
...     ...       ...          ...
217       4  0.015183         work
182       1  0.358314     workflow
182       3  0.536167     workflow
182       4  0.105526     workflow
183       1  0.997139        wrong

[219 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 4, 7, 5, 3, 2])